In [4]:
import random, pygame
from time import sleep, time
from pygame.locals import *
WINDOW_WIDTH = 480
WINDOW_HEIGHT = 640
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (250, 250, 50)
YELLOW_2 = (244, 244, 76)
RED = (250, 50, 50)
FPS = 60

class Fighter(pygame.sprite.Sprite):
    def __init__(self):
        super(Fighter, self).__init__()
        self.image = pygame.image.load('fighter.png')
        self.rect = self.image.get_rect()
        self.rect.x = int(WINDOW_WIDTH / 2)
        self.rect.y = WINDOW_HEIGHT - self.rect.height
        self.dx = 0
        self.dy = 0
        
    def update(self):
        self.rect.x += self.dx
        self.rect.y += self.dy
        if self.rect.x < 0 or self.rect.x + self.rect.width > WINDOW_WIDTH:
            self.rect.x -= self.dx
        if self.rect.y < 0 or self.rect.y + self.rect.height > WINDOW_HEIGHT:
            self.rect.y -= self.dy
    
    def draw(self, screen):
        screen.blit(self.image, self.rect)
    
    def collide(self, sprites):
        for sprite in sprites:
            if pygame.sprite.collide_rect(self, sprite):
                return sprite
            
class Missile(pygame.sprite.Sprite):
    def __init__(self, xpos, ypos, speed):
        super(Missile, self).__init__()
        self.image = pygame.image.load('missile.png')
        self.rect = self.image.get_rect()
        self.rect.x = xpos
        self.rect.y = ypos
        self.speed = speed
        self.sound = pygame.mixer.Sound('missile.wav')
    
    def launch(self):
        self.sound.play()
    
    def update(self):
        self.rect.y -= self.speed
        if self.rect.y + self.rect.height < 0 :
            self.kill()
    
    def collide(self, sprites):
        for sprite in sprites:
            if pygame.sprite.collide_rect(self, sprite):
                return sprite
            
class Enemy(pygame.sprite.Sprite):
    def __init__(self, xpos, ypos, speed):
        super(Enemy, self).__init__()
        enemy_images = ('animal1.png', 'animal2.png', 'animal3.png', 'animal4.png', 'animal5.png',\
                         'rock1.png', 'rock2.png', 'rock3.png', 'rock4.png', 'rock5.png', \
                         'rock6.png', 'rock7.png', 'rock8.png', 'rock9.png', 'rock10.png', \
                         'alien1.png', 'alien2.png', 'alien3.png', 'alien4.png', \
                         'spaceship1.png', 'spaceship2.png', 'spaceship3.png', 'spaceship4.png')
        self.image = pygame.image.load(random.choice(enemy_images))
        self.rect = self.image.get_rect()
        self.rect.x = xpos
        self.rect.y = ypos
        self.speed = speed
    
    def update(self):
        self.rect.y += self.speed
    
    def out_of_screen(self):
        if self.rect.y > WINDOW_HEIGHT:
            return True
        
def draw_text(text, font, surface, x, y, main_color):
    text_obj = font.render(text, True, main_color)
    text_rect = text_obj.get_rect()
    text_rect.centerx = x
    text_rect.centery = y
    surface.blit(text_obj, text_rect)
   
    
def occur_explosion(surface, x, y):
    explosion_image = pygame.image.load('explosion.png')
    explosion_rect = explosion_image.get_rect()
    explosion_rect.x = x
    explosion_rect.y = y
    surface.blit(explosion_image, explosion_rect)
    explosion_sounds = ('explosion01.mp3', 'explosion02.wav', 'explosion03.wav')
    explosion_sound = pygame.mixer.Sound(random.choice(explosion_sounds))
    explosion_sound.play()

def intro():
    start_image = pygame.image.load('background.png')  
    screen.blit(start_image, [0,0])
    draw_x = int(WINDOW_WIDTH / 2)
    draw_y = int(WINDOW_HEIGHT / 4)
    font_20 = pygame.font.Font('PFStardust.ttf', 20, Bold = True)
    
    draw_text('지구로부터 멀리 떨어진 달나라에서', font_20, screen, draw_x, draw_y, WHITE)
    draw_text('토끼들이 평화롭게 살고 있었습니다...', font_20, screen, draw_x, draw_y + 50, WHITE)
    draw_text('그러던 어느날, 초신성의 대폭발과 함께', font_20, screen, draw_x , draw_y + 150, WHITE)
    draw_text('블랙홀에서 맹수 무리와 외계인들이 넘어왔습니다!', font_20, screen, draw_x, draw_y + 200, WHITE)
    draw_text('맹수 무리와 외계인, 그리고 폭발 잔해로부터', font_20, screen, draw_x, draw_y + 300, WHITE)
    draw_text('달나라를 지켜주세요', font_20, screen, draw_x, draw_y + 350, WHITE)
    
    pygame.display.update()
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                return 'game_menu'
        if event.type == QUIT:
            return 'quit'
    return 'intro'    
    
def game_menu():
    start_image = pygame.image.load('main.png')
    screen.blit(start_image, [0,0])
    draw_x = int(WINDOW_WIDTH / 2)
    draw_y = int(WINDOW_HEIGHT / 4)
    font_60 = pygame.font.Font('PFStardust.ttf', 60, Bold=True)
    font_30 = pygame.font.Font('PFStardust.ttf', 30)
    font_2 = pygame.font.Font('bolsa_m.ttf', 20)
    
    draw_text('달나라를 지켜라!', font_60, screen, draw_x, draw_y, YELLOW_2)
    draw_text('엔터 키를 누르면', font_30, screen, draw_x, draw_y +250, BLACK)
    draw_text('게임이 시작됩니다', font_30, screen, draw_x, draw_y +300, BLACK)
    draw_text('공격 : z   스킬1 : x   스킬 : SPACE', font_2, screen, draw_x, draw_y +400, BLACK)
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                return 'play'
        if event.type == QUIT:
            return 'quit'
    return 'game_menu'

def outro():    
    screen_r = screen.get_rect()
    font = pygame.font.Font('PFStardust.ttf', 20, Bold = True)
    clock = pygame.time.Clock()
    
    credit_list = ["만든 사람들"," ","5ZGO렛잇고", " ", "민균 :)", "준서 ^^", "유림 :D", "원익 ><", ' ', 'Thanks!']

    texts = []
    
    for i, line in enumerate(credit_list):
        s = font.render(line, 1, WHITE)
        r = s.get_rect(centerx=screen_r.centerx, y=screen_r.bottom + i * 45)
        texts.append((r, s))

    while True:
        for e in pygame.event.get():
            if e.type == QUIT or e.type == KEYDOWN and e.key == pygame.K_RETURN:
                return 'game_over'

        screen.fill((0, 0, 0))

        for r, s in texts:
            r.move_ip(0, -1)
            screen.blit(s, r)
        
        pygame.display.flip()
        clock.tick(60)
            
        if not screen_r.collidelistall([r for (r, _) in texts]):
            return 'black_fill'

def Black_fill():
    screen_r = screen.get_rect()
    clock = pygame.time.Clock()
    while True:
        screen.fill((0, 0, 0))
        pygame.display.flip()
        clock.tick(60)
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    return 'game_over'
            if event.type in [pygame.QUIT]:
                return 'quit'
        
def game_over():
    start_image = pygame.image.load('main.png')
    screen.blit(start_image, [0,0])
    draw_x = int(WINDOW_WIDTH / 2)
    draw_y = int(WINDOW_HEIGHT / 4)
    font_50 = pygame.font.Font('PFStardust.ttf', 50)
    font_30 = pygame.font.Font('PFStardust.ttf', 30)
    
    draw_text('GAME OVER', font_50, screen, draw_x, draw_y, YELLOW_2)
    draw_text('엔터 키를 누르면', font_30, screen, draw_x, draw_y +250, BLACK)
    draw_text('게임이 다시 시작됩니다', font_30, screen, draw_x, draw_y +300, BLACK)
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                return 'play'
        if event.type == QUIT:
            return 'quit'
    return 'game_over'

def game_loop():
    default_font = pygame.font.Font('PFStardust.ttf', 28)
    background_image = pygame.image.load('background.png')
    background_image2 = pygame.image.load('background2.png')
    background_image3 = pygame.image.load('background3.png')
    gameover_sound = pygame.mixer.Sound('gameover.wav')
    pygame.mixer.music.load('music.wav')
    pygame.mixer.music.play(-1)
    fps_clock = pygame.time.Clock()
    fighter = Fighter()
    missiles = pygame.sprite.Group()
    enemies = pygame.sprite.Group()
    occur_prob = 40
    shot_count = 0
    count_missed = 0
    x = 0
    t = 0
    x1 = 3
    x2 = 1
    
    done = False
    while not done:
        for event in pygame.event.get():
            if event.type in [pygame.QUIT]:
                return 'quit'
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    fighter.dx -= 5
                elif event.key == pygame.K_RIGHT:
                    fighter.dx += 5
                elif event.key == pygame.K_UP:
                    fighter.dy -= 5
                elif event.key == pygame.K_DOWN:
                    fighter.dy += 5
                elif event.key == pygame.K_z:
                    for i in range(-20, 21, 20):
                        missile = Missile(fighter.rect.centerx + i, fighter.rect.y, 10)
                        missile.launch()
                        missiles.add(missile)
                elif event.key == pygame.K_x:
                    if x1 >0:
                        x1 -= 1
                        for i in range(-100,120,20):
                            if i > 0:
                                missile = Missile(fighter.rect.centerx + i, fighter.rect.y + i, 10)
                                missile.launch()
                                missiles.add(missile)
                            else:
                                missile = Missile(fighter.rect.centerx + i, fighter.rect.y - i, 10)
                                missile.launch()
                                missiles.add(missile)
                elif event.key == pygame.K_SPACE:
                    if x2 > 0:
                        x2 -= 1
                        t = time()
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                    fighter.dx = 0
                elif event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                    fighter.dy = 0
        
        if shot_count < 20:
            screen.blit(background_image, background_image.get_rect())
            level = 1
        elif shot_count < 50:
            screen.blit(background_image2, background_image.get_rect())
            level = 2
        else:
            screen.blit(background_image3, background_image.get_rect())
            level = 3
        occur_of_enemies = 1 + int(shot_count / 300)
        min_ene_speed = 1 + int(shot_count / 200)
        max_ene_speed = 1 + int(shot_count / 100)
        
        if random.randint(1, occur_prob) == 1:
            for i in range(occur_of_enemies):
                speed = random.randint(min_ene_speed, max_ene_speed)
                enemy = Enemy(random.randint(0, WINDOW_WIDTH - 30), 0, speed)
                enemies.add(enemy)
        draw_text('SCORE: {}'.format(shot_count), default_font, screen, 100, 20, YELLOW)
        draw_text('LEVEL: {}'.format(level), default_font, screen, 99, 50, WHITE)
        draw_text('MISSED: {}'.format(count_missed), default_font, screen, 380, 20, RED)
        font_20 = pygame.font.Font('PFStardust.ttf', 20)
        draw_text('남은 필살기: {}'.format(x1), font_20, screen, 68, 620, RED)
        draw_text('남은 무적: {}'.format(x2), font_20, screen, 60, 580, RED)
        
        if t + 5 > time():
            font_20 = pygame.font.Font('PFStardust.ttf', 20)
            draw_text('무적시간 : {}'.format(str(round(time() - t, 1))), font_20, screen, 400, 620, RED)
        
        for missile in missiles:
            enemy = missile.collide(enemies)
            if enemy:
                missile.kill()
                if shot_count < 25:
                    enemy.kill()
                    occur_explosion(screen, enemy.rect.x, enemy.rect.y)
                    shot_count += 1
                elif x < 1:
                    x += 1
                else:
                    enemy.kill()
                    occur_explosion(screen, enemy.rect.x, enemy.rect.y)
                    shot_count += 1
                    x=0
        
        for enemy in enemies:
            if enemy.out_of_screen():
                enemy.kill()
                if t + 5 < time():
                    count_missed += 1
        
        enemies.update()
        enemies.draw(screen)
        missiles.update()
        missiles.draw(screen)
        fighter.update()
        fighter.draw(screen)
        pygame.display.flip()
        
        if fighter.collide(enemies) or count_missed >= 3:
            if t + 6 < time():
                pygame.mixer_music.stop()
                occur_explosion(screen, fighter.rect.x, fighter.rect.y)
                pygame.display.update()
                gameover_sound.play()
                sleep(1)
                done = True
        fps_clock.tick(FPS)
    return 'outro'

def main():
    global screen
    pygame.init()
    screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
    pygame.display.set_caption('MOON RABBIT')
    icon_image = pygame.image.load('rabbit_icon.png')
    pygame.display.set_icon(icon_image)
    
    action = 'intro'
    while action != 'quit':
        if action == 'intro':
            action = intro()
        elif action == 'game_menu':
            action = game_menu()
        elif action == 'play':
            action = game_loop()
        elif action == 'outro':
            action = outro()
        elif action == 'game_over':
            action = game_over()
        elif action == 'black_fill':
            action = Black_fill()
    pygame.quit()
    
if __name__ == "__main__":
    main()

In [2]:
pip install pygame

     |████████████████████████████████| 8.9 MB 8.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
